
#BL Generation : P-2-A : AUH1



#Default Labels

In [ ]:
import pandas as pd

# section mapping
section_ranges = {
    4: (200, 290),
    3: (320, 430),
    2: (460, 550),
    1: (580, 660)
}


def labels_for_zone(floor, zone, start_aisle, end_aisle, shelves, boxes_per_shelf, sections):
    # Constants
    max_shelves = 9
    color_codes = ['FFFFFF', '339900', '9B30FF', 'FFFF00', '00FFFF', 'CC0000', 'F88017', 'FF00FF', '996600']
    bay_increment = 10  # Number of bays between each section

    # Initialize DataFrame to store all labels
    columns = ['AISLE', 'SLOT'] + [f"{color}({chr(65 + i)})" for i, color in enumerate(color_codes)]
    all_data = []

    # Generate labels for each specified aisle
    for aisle in range(start_aisle, end_aisle + 1):
        for section in sections:
            start_bay, end_bay = section_ranges[section]

            # Generate labels for each bay in the current section within the aisle
            for bay in range(start_bay, end_bay + 1, bay_increment):
                current_slot = bay  # Assuming SLOT corresponds directly to bay for simplicity
                bay_data = {col: [] for col in columns}

                # Populate labels for each shelf in the bay
                for box in range(boxes_per_shelf):
                    row = {'AISLE': aisle, 'SLOT': current_slot}
                    for j in range(max_shelves):
                        if j < shelves:
                            label = f"{floor}-{zone}{aisle}{chr(65+j)}{current_slot + box}"
                            row[f"{color_codes[j]}({chr(65+j)})"] = label
                        else:
                            row[f"{color_codes[j]}({chr(65+j)})"] = 'X'
                    for col in columns:
                        bay_data[col].append(row[col])

                # Convert bay data to DataFrame and append to all data
                df_bay = pd.DataFrame(bay_data)
                all_data.append(df_bay)

    # Concatenate all bays into a single DataFrame
    df_all_aisles = pd.concat(all_data, ignore_index=True)
    return df_all_aisles



#Remove Labels according to Bay Configuration

In [ ]:
def remove_labels(df, floor, zone, start_aisle, end_aisle, aisle_jump, shelf_box_info, sections, bays):
    # Extract specific shelf and maximum boxes information

    color_codes = ['FFFFFF', '339900', '9B30FF', 'FFFF00', '00FFFF', 'CC0000', 'F88017', 'FF00FF', '996600']

    shelf_boxes = {}
    for info in shelf_box_info.split(','):
        shelf, max_boxes = info.split(':')
        shelf_boxes[shelf.strip()] = int(max_boxes)

    # Filter rows based on the aisle range, aisle jump, and sections
    for section in sections:
        start_bay, end_bay = section_ranges[section]
        for aisle in range(start_aisle, end_aisle + 1, aisle_jump):
            for bay in range(start_bay, min(end_bay, start_bay + bays * 10), 10):
                # Filter the DataFrame for the specific aisles and bays
                df_filtered = df[(df['AISLE'] == aisle) & (df['SLOT'] == bay)]

                # Adjust the labels for the specified shelves
                for index, row in df_filtered.iterrows():
                    for shelf, max_boxes in shelf_boxes.items():
                        # Determine which columns to adjust based on the shelf
                        col_name = f"{color_codes[ord(shelf) - 65]}({shelf})"
                        for box_index in range(max_boxes, 6):  # Assuming the boxes start from 0 to 5
                            # Construct the label to find
                            label_to_replace = f"{floor}-{zone}{aisle}{shelf}{bay + box_index}"
                            if row[col_name] == label_to_replace:
                                df.loc[index, col_name] = 'X'


#Remove Labels for Obstructions


In [ ]:
import pandas as pd

def parse_input(input_value):
    """ Helper function to parse input values into a list of integers. Handles ranges and comma-separated values. """
    result = []
    parts = input_value.split(',')
    for part in parts:
        if '-' in part:
            start, end = map(int, part.split('-'))
            result.extend(range(start, end + 1))
        else:
            result.append(int(part))
    return result

def mark_obstructed_bays(df, floor, zone, aisles, bays):
    aisles = parse_input(aisles)
    bays = parse_input(bays)

    # Iterate over the DataFrame and replace bin labels for obstructed bays
    for index, row in df.iterrows():
        if row['AISLE'] in aisles and row['SLOT'] in bays:
            for col in df.columns:
                if col not in ['AISLE', 'SLOT']:  # Avoid altering aisle and slot columns
                    if f"{floor}-{zone}" in row[col]:  # Check if the floor-zone prefix is in the label
                        df.at[index, col] = 'X'

    return df



#Inputs

In [ ]:
#default labels
original_df = labels_for_zone("P2", "A", 200, 262, 8, 6, [4,3,2,1])

#remove labels acc to bin configuration
remove_labels(original_df, "P2", "A", 200, 254, 2, "A:4,B:4,C:4,D:4,E:4,F:4,G:4,H:4", [4, 3, 2], 2)
remove_labels(original_df, "P2", "A", 208, 254, 1, "D:2,E:2,F:2", [1], 3)
remove_labels(original_df, "P2", "A", 201, 253, 1, "A:4,B:4,C:4,D:5,E:5,F:4,G:4,H:4", [4, 3, 2], 2)
remove_labels(original_df, "P2", "A", 255, 262, 1, "A:4,B:4,C:4,D:4,E:4,F:4,G:4,H:4", [4, 3, 2], 2)

#remove labels for obstructed bays
original_df = mark_obstructed_bays(original_df, 'P2', 'A', '223-226', '250, 270, 270')# for general walkways in mod1
original_df = mark_obstructed_bays(original_df, 'P2', 'A', '201-214', '350')# for general walkways in mod2
original_df = mark_obstructed_bays(original_df, 'P2', 'A', '211-212', '340')# for general walkways in mod2
original_df = mark_obstructed_bays(original_df, 'P2', 'A', '211-214', '390')# for general walkways in mod2
original_df = mark_obstructed_bays(original_df, 'P2', 'A', '231-232', '410')# for general walkways in mod2
original_df = mark_obstructed_bays(original_df, 'P2', 'A', '241-242', '340')# for general walkways in mod2
original_df = mark_obstructed_bays(original_df, 'P2', 'A', '243-244', '430')# for general walkways in mod2
original_df = mark_obstructed_bays(original_df, 'P2', 'A', '201-208, 211-212, 223-226, 241-242', '600')# for general walkways in mod4
original_df = mark_obstructed_bays(original_df, 'P2', 'A', '223-226', '620')# for general walkways in mod4
original_df = mark_obstructed_bays(original_df, 'P2', 'A', '237-238', '640')# for general walkways in mod4
original_df = mark_obstructed_bays(original_df, 'P2', 'A', '224-225', '200-240')# stair1
original_df = mark_obstructed_bays(original_df, 'P2', 'A', '212-213', '400-430')# stair2
original_df = mark_obstructed_bays(original_df, 'P2', 'A', '224-225', '630-660')# stair3
original_df = mark_obstructed_bays(original_df, 'P2', 'A', '200-211', '200-330')# storage not used
original_df = mark_obstructed_bays(original_df, 'P2', 'A', '200-207', '630-660')# storage not used


display(original_df)


,AISLE,SLOT,FFFFFF(A),339900(B),9B30FF(C),FFFF00(D),00FFFF(E),CC0000(F),F88017(G),FF00FF(H),996600(I)
0,200,200,X,X,X,X,X,X,X,X,X
1,200,200,X,X,X,X,X,X,X,X,X
2,200,200,X,X,X,X,X,X,X,X,X
3,200,200,X,X,X,X,X,X,X,X,X
4,200,200,X,X,X,X,X,X,X,X,X
...,...,...,...,...,...,...,...,...,...,...,...
15493,262,660,P2-A262A661,P2-A262B661,P2-A262C661,P2-A262D661,P2-A262E661,P2-A262F661,P2-A262G661,P2-A262H661,X
15494,262,660,P2-A262A662,P2-A262B662,P2-A262C662,P2-A262D662,P2-A262E662,P2-A262F662,P2-A262G662,P2-A262H662,X
15495,262,660,P2-A262A663,P2-A262B663,P2-A262C663,P2-A262D663,P2-A262E663,P2-A262F663,P2-A262G663,P2-A262H663,X
15496,262,660,P2-A262A664,P2-A262B664,P2-A262C664,P2-A262D664,P2-A262E664,P2-A262F664,P2-A262G664,P2-A262H664,X


#Get Excel Data File

In [ ]:
import pandas as pd

# Convert DataFrame to Excel with a specific sheet name
file_path = '/content/BL.xlsx'
original_df.to_excel(file_path, index=False, sheet_name='P2A')

# Download the file to your local system
from google.colab import files
files.download(file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>